Demo code for real-time staging using HRV 

(c)-2020 Neurobit Technologies Pte Ltd Singapore  
https://www.neurobit.io  
protected by copyright law and international treaties  
Licensed under Neurobit End User License Agreement (EULA)  
Strictly for academic, research and non-commercial use only  
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR  
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,  
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE  
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER  
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,  
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN  
THE SOFTWARE.  

In [101]:
import pyedflib
import numpy as np
from requests import post
from pyndf import create_ndf_from_ecg
from sklearn.metrics import cohen_kappa_score

In [102]:
# Login information for Z3Score
# Note that real-time module is only available to select customers
# Always use https instead of http to ensure security
server_url = 'https://z3score.com/api/v4'
# Do not have a key? Request one from contact@neurobit.io
email = 'emailid'
key = r'yourKey'

In [103]:
# You need an authorization token to access the real-time module
# Use your login details to request it. The token auto expires after 16 hours
# Log in is a slow process, authorization tokens are a fast way to allow access to services 
# Note: you need special previlages to access the real-time module
try:
    response = post(server_url+'/get-token', data={'email':email, 'key':key})
except:
    print("Error communicating with server")
    exit(0)

if response.status_code != 200:
    print("Error communicating with server")
    exit(0)

data = response.json()
token = data['token']

if data['token'] == 0:
    print("Could not generate access token")
    print(data['message'])
    exit(0)

print("Access token generated for real-time module...")
print(data['message'])

Access token generated for real-time module...
Token is valid for 16 hours only.


In [104]:
path = 'sample.edf'
edf_file = pyedflib.EdfReader(path)
labels = edf_file.getSignalLabels()
print("Here are the channel labels:")
for idx, label in enumerate(labels):
    print('%d. %s' % (idx, label))

Here are the channel labels:
0. C3-M2
1. C4-M1
2. O1-M2
3. O2-M1
4. F3-M2
5. F4-M1
6. Chin1-Chin2
7. EKG
8. PG1-M2
9. PG2-M1


In [105]:
# channel 7 is EKG
fs = edf_file.getSampleFrequency(7)
ECG = np.asarray(edf_file.readSignal(7))
total_epochs = int(np.shape(ECG)[0]/fs/30)
edf_file._close()

In [113]:
# Lets Load the expert scores 
import csv
with open('expert_scores.csv', 'rt') as f:
    expert = sum([[int(x) for x in rec] for rec in csv.reader(f, delimiter=',')], [])

expert = np.array(expert)
# 9 is unscorable which we can assume wake
expert[expert==9] = 0
# N1 is light sleep 
expert[expert==1] = 2

In [118]:
stage_labels = {
    0:'Wake',
    2:'Light',
    3:'Deep',
    5:'REM'
}
# you need 9 epochs or 4.5 minutes of data to access the realtime staging
for i in range(8,total_epochs):
    stream = create_ndf_from_ecg(ECG[(i-8)*30*fs:(i+1)*30*fs], fs, realtime=True)
    files = {'file': ('stream.ndf', stream)}
    response = post(server_url + '/realtime/hrv', files=files, data={'token':token})
    response = response.json()
    stage = response['message']
    print('Predicted Stage: %s, Confidence: %0.2f, Expert Stage: %s' %(stage_labels[stage[0]], stage[1], stage_labels[expert[i]]))

Predicted Stage: Wake, Confidence: 0.21, Expert Stage: Light
Predicted Stage: Wake, Confidence: 2.20, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.60, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.47, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.37, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.49, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.78, Expert Stage: Light
Predicted Stage: Light, Confidence: 1.12, Expert Stage: Light
Predicted Stage: Deep, Confidence: 0.20, Expert Stage: Light
Predicted Stage: Wake, Confidence: 1.20, Expert Stage: Wake
Predicted Stage: Light, Confidence: 0.33, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.37, Expert Stage: Light
Predicted Stage: Light, Confidence: 1.46, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.89, Expert Stage: Light
Predicted Stage: Light, Confidence: 2.22, Expert Stage: Light
Predicted Stage: Light, Confidence: 0.32, Expert Stage: Light
Predicted Sta